In [ ]:
import sys,os
sys.path.insert(1,'/rpod3/lsiemens/offline_OMEGA_SYGMA/OMEGA_SYGMA_critter')
os.environ["SYGMADIR"] = "/rpod3/lsiemens/offline_OMEGA_SYGMA/OMEGA_SYGMA_critter"
import matplotlib
%matplotlib inline

from modules.SYGMA import start_SYGMA

In [ ]:
start_SYGMA()